# 첫 번째 캠페인 만들기

이 노트북에서는 Movielens 데이터 세트에서 수집된 데이터를 기반으로 영화에 대한 추천 모델을 구축하는 단계를 안내합니다. 목표는 특정 사용자를 기준으로 연관성 높은 영화를 추천하는 것입니다.

데이터는 MovieLens 프로젝트에서 가져오며, 아래 셀에서 기다리는 시간 동안 웹 검색을 통해 데이터와 그 잠재적 사용 사례를 알아볼 수 있습니다.

## 이 노트북을 사용하는 방법

코드는 아래와 같이 셀로 구분됩니다. 이 페이지 맨 위에 삼각형 모양의 `Run` 버튼이 있습니다. 이 버튼을 클릭하여 각 셀을 실행하고 다음 셀로 이동하거나, 셀에 있는 동안 `Shift` + `Enter`를 눌러 셀을 실행한 후 다음 셀로 이동할 수 있습니다.

셀이 실행되면, 셀이 실행되는 동안에는 옆의 라인에 `*`가 표시되며, 셀 내의 모든 코드 실행을 완료하고 나면 이 기호가 실행을 완료한 마지막 셀을 나타내는 숫자로 업데이트됩니다.


아래 지침에 따라 셀을 실행하여 Amazon Personalize를 시작하세요.

## 가져오기 

Python은 다양한 라이브러리 컬렉션과 함께 제공되며, 핵심 데이터 과학 도구로서 설치된 [boto3](https://aws.amazon.com/sdk-for-python/)(python용 AWS SDK), [Pandas](https://pandas.pydata.org/)/[Numpy](https://numpy.org/) 등을 라이브러리와 함께 가져와야 합니다.

In [ ]:
# Imports
import boto3
import json
import numpy as np
import pandas as pd
import time
!conda install -y -c conda-forge unzip

그런 다음 사용자 환경이 Amazon Personalize와 성공적으로 통신할 수 있는지 검증해야 합니다. 아래 코드 줄은 이 작업을 수행합니다.

In [ ]:
# Configure the SDK to Personalize:
personalize = boto3.client('personalize')
personalize_runtime = boto3.client('personalize-runtime')

## 데이터 구성

데이터는 Amazon S3를 통해 Amazon Personalize로 가져옵니다. 아래에서는 이 연습을 위해 AWS 내에서 생성한 버킷을 지정합니다.

아래에서는 CloudFormation 단계에서 이전에 생성한 값으로 설정되도록 `bucket` 변수를 업데이트합니다. 이 값은 이전 작업의 텍스트 파일에 있습니다. `filename`은 변경할 필요가 없습니다.

### 버킷 및 데이터 출력 위치 지정
CloudFormation 템플릿을 배포하는 동안 값을 사용자 지정한 경우 `bucket` 값을 업데이트해야 합니다. 아래 셀을 클릭하여 변경합니다.

In [ ]:
bucket = "personalizedemofirstnamelastname"       # replace with the name of your S3 bucket
filename = "movie-lens-100k.csv"

### 훈련 데이터 다운로드, 준비 및 업로드

현재 MovieLens 데이터가 아직 로컬로 로드되지 않았습니다. 아래 코드 줄을 실행하여 최신 복사본을 다운로드하고 간단히 검사합니다.

#### 데이터 세트 다운로드 및 탐색

In [ ]:
!wget -N http://files.grouplens.org/datasets/movielens/ml-100k.zip
!unzip -o ml-100k.zip
data = pd.read_csv('./ml-100k/u.data', sep='\t', names=['USER_ID', 'ITEM_ID', 'RATING', 'TIMESTAMP'])
pd.set_option('display.max_rows', 5)
data

#### 데이터 준비 및 업로드

보시다시피, 이 데이터에는 UserID, ItemID, 평점 및 타임스탬프가 포함됩니다.

이제 낮은 순위의 항목을 제거하고, 모델을 만들기 전에 평가 열을 제거합니다.

작업이 완료되면 파일을 새 CSV로 저장한 다음 S3에 업로드합니다.

이 작업은 모두 아래 셀의 코드 줄을 실행하여 수행합니다.

In [ ]:
data = data[data['RATING'] > 3]                # Keep only movies rated higher than 3 out of 5.
data = data[['USER_ID', 'ITEM_ID', 'TIMESTAMP']] # select columns that match the columns in the schema below
data.to_csv(filename, index=False)
boto3.Session().resource('s3').Bucket(bucket).Object(filename).upload_file(filename)

### 스키마 생성

Personalize가 데이터를 이해하기 위한 핵심 구성 요소는 아래에 정의된 스키마에서 얻어집니다. 이 구성은 Personalize 서비스에 CSV 파일을 통해 제공된 데이터를 수집하는 방법을 알려줍니다. 열과 유형은 위에서 만든 파일에 있는 내용과 일치합니다.

In [ ]:
schema = {
    "type": "record",
    "name": "Interactions",
    "namespace": "com.amazonaws.personalize.schema",
    "fields": [
        {
            "name": "USER_ID",
            "type": "string"
        },
        {
            "name": "ITEM_ID",
            "type": "string"
        },
        {
            "name": "TIMESTAMP",
            "type": "long"
        }
    ],
    "version": "1.0"
}

create_schema_response = personalize.create_schema(
    name = "personalize-demo-schema",
    schema = json.dumps(schema)
)

schema_arn = create_schema_response['schemaArn']
print(json.dumps(create_schema_response, indent=2))

### 데이터 세트 그룹 생성 및 생성이 완료될 때까지 기다리기

Personalize에서 가장 큰 그룹은 데이터 세트 그룹으로, 데이터, 이벤트 트래커, 솔루션 및 캠페인을 격리합니다. 공통적인 데이터 컬렉션을 공유하는 것들끼리 한데 그룹화합니다. 원하는 경우 아래의 이름을 자유롭게 변경해도 됩니다.

#### 데이터 세트 그룹 생성

In [ ]:
create_dataset_group_response = personalize.create_dataset_group(
    name = "personalize-launch-demo"
)

dataset_group_arn = create_dataset_group_response['datasetGroupArn']
print(json.dumps(create_dataset_group_response, indent=2))

#### 데이터 세트 그룹이 활성 상태가 될 때까지 기다리기

아래의 항목에서 데이터 세트 그룹을 사용하려면 먼저 아래의 셀을 실행하고 활성 상태로 표시될 때까지 기다립니다.

In [ ]:
max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_dataset_group_response = personalize.describe_dataset_group(
        datasetGroupArn = dataset_group_arn
    )
    status = describe_dataset_group_response["datasetGroup"]["status"]
    print("DatasetGroup: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)

#### 데이터 세트 생성

그룹을 생성한 후에는 실제 데이터 세트를 만들어야 합니다. 이 예에서는 상호 작용 데이터의 데이터 세트를 하나만 만듭니다. 아래의 셀을 실행하여 데이터 세트를 만듭니다.

In [ ]:
dataset_type = "INTERACTIONS"
create_dataset_response = personalize.create_dataset(
    name = "personalize-launch-interactions",
    datasetType = dataset_type,
    datasetGroupArn = dataset_group_arn,
    schemaArn = schema_arn
)

dataset_arn = create_dataset_response['datasetArn']
print(json.dumps(create_dataset_response, indent=2))

#### S3 버킷에 정책 연결

앞서 생성한 S3 버킷의 콘텐츠를 Amazon Personalize가 읽을 수 있어야 합니다. 아래의 코드 줄은 이를 가능하게 합니다.

In [ ]:
s3 = boto3.client("s3")

policy = {
    "Version": "2012-10-17",
    "Id": "PersonalizeS3BucketAccessPolicy",
    "Statement": [
        {
            "Sid": "PersonalizeS3BucketAccessPolicy",
            "Effect": "Allow",
            "Principal": {
                "Service": "personalize.amazonaws.com"
            },
            "Action": [
                "s3:GetObject",
                "s3:ListBucket"
            ],
            "Resource": [
                "arn:aws:s3:::{}".format(bucket),
                "arn:aws:s3:::{}/*".format(bucket)
            ]
        }
    ]
}

s3.put_bucket_policy(Bucket=bucket, Policy=json.dumps(policy))

#### Personalize 역할 생성

또한 Amazon Personalize가 특정 작업을 실행할 수 있는 권한을 가지려면, AWS에서 역할을 수임할 수 있어야 합니다. 아래의 코드 줄은 이 권한을 부여합니다.

In [ ]:
iam = boto3.client("iam")

role_name = "PersonalizeRoleDemo"
assume_role_policy_document = {
    "Version": "2012-10-17",
    "Statement": [
        {
          "Effect": "Allow",
          "Principal": {
            "Service": "personalize.amazonaws.com"
          },
          "Action": "sts:AssumeRole"
        }
    ]
}

create_role_response = iam.create_role(
    RoleName = role_name,
    AssumeRolePolicyDocument = json.dumps(assume_role_policy_document)
)

# AmazonPersonalizeFullAccess provides access to any S3 bucket with a name that includes "personalize" or "Personalize" 
# if you would like to use a bucket with a different name, please consider creating and attaching a new policy
# that provides read access to your bucket or attaching the AmazonS3ReadOnlyAccess policy to the role
policy_arn = "arn:aws:iam::aws:policy/service-role/AmazonPersonalizeFullAccess"
iam.attach_role_policy(
    RoleName = role_name,
    PolicyArn = policy_arn
)

# Now add S3 support
iam.attach_role_policy(
    PolicyArn='arn:aws:iam::aws:policy/AmazonS3FullAccess',
    RoleName=role_name
)
time.sleep(60) # wait for a minute to allow IAM role policy attachment to propagate

role_arn = create_role_response["Role"]["Arn"]
print(role_arn)

## 데이터 가져오기

앞서 데이터 세트 그룹과 데이터 세트를 만들어 정보를 저장했으므로, 이제 모델 구축 사용 사례를 위해 S3에서 Amazon Personalize로 데이터를 로드하는 가져오기 작업을 실행합니다.

#### 데이터 세트 가져오기 작업 생성

In [ ]:
create_dataset_import_job_response = personalize.create_dataset_import_job(
    jobName = "personalize-demo-import1",
    datasetArn = dataset_arn,
    dataSource = {
        "dataLocation": "s3://{}/{}".format(bucket, filename)
    },
    roleArn = role_arn
)

dataset_import_job_arn = create_dataset_import_job_response['datasetImportJobArn']
print(json.dumps(create_dataset_import_job_response, indent=2))

#### 데이터 세트 가져오기 작업이 활성 상태가 될 때까지 기다리기

가져오기 작업이 완료되는 데 시간이 걸릴 수 있습니다. 아래에 활성 상태로 표시될 때까지 기다리세요.

In [ ]:
max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_dataset_import_job_response = personalize.describe_dataset_import_job(
        datasetImportJobArn = dataset_import_job_arn
    )
    status = describe_dataset_import_job_response["datasetImportJob"]['status']
    print("DatasetImportJob: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)

## 솔루션 및 버전 생성

Amazon Personalize에서는 훈련된 모델을 솔루션이라고 하며, 각 솔루션은 모델을 훈련할 때 주어진 데이터 볼륨과 관련된 여러 버전을 가질 수 있습니다.

먼저 지원되는 모든 레시피를 나열합니다. 레시피는 데이터를 아직 훈련하지 않은 알고리즘입니다. 목록을 작성한 후 하나를 선택하여 모델을 구축하는 데 사용할 수 있습니다.

### 레시피 선택

In [ ]:
list_recipes_response = personalize.list_recipes()
list_recipes_response

#### 사용자 개인화
[사용자 개인화](https://docs.aws.amazon.com/personalize/latest/dg/native-recipe-new-item-USER_PERSONALIZATION.html)(aws-user-personalization) 레시피는 모든 USER_PERSONALIZATION 추천 시나리오에 최적화된 레시피입니다. 이 레시피는 항목을 추천할 때 자동 항목 탐색을 사용합니다.

자동 탐색 기능을 통해 Amazon Personalize는 자동으로 여러 가지 항목 추천을 테스트하고, 사용자가 이러한 추천 항목과 상호 작용하는 방법을 학습하며, 더 나은 참여와 전환을 유도하는 항목에 대한 추천을 강화합니다. 이렇게 하면 카탈로그가 빠르게 변경되거나 뉴스 기사 또는 프로모션과 같은 새 항목이 새것인 상태일 때 사용자에게 더 연관성이 높은 경우, 항목 검색률과 참여율이 향상됩니다.

탐색할 양(상호 작용 데이터 또는 연관성이 적은 항목이 더 자주 추천되는 경우)과 활용할 양(추천이 우리가 알고 있는 내용 또는 연관성을 기반으로 하는 경우)을 균형 있게 조정할 수 있습니다. Amazon Personalize는 암시적인 사용자 피드백에 따라 향후 추천을 자동으로 조정합니다.

먼저 위의 레시피 목록에서 ARN을 찾아 레시피를 선택합니다.

In [ ]:
recipe_arn = "arn:aws:personalize:::recipe/aws-user-personalization" # aws-user-personalization selected for demo purposes

### 솔루션 생성 및 기다리기

먼저 API로 솔루션을 생성한 다음 버전을 생성합니다. 모델을 훈련하고 해당 버전의 솔루션을 만드는 데 몇 분 정도 걸립니다. 이 프로세스가 시작되고 진행률 알림이 표시되면, 잠시 쉬거나 커피를 마시는 등의 시간으로 활용할 수 있습니다.

#### 솔루션 생성

In [ ]:
create_solution_response = personalize.create_solution(
    name = "personalize-demo-soln-user-personalization",
    datasetGroupArn = dataset_group_arn,
    recipeArn = recipe_arn
)

solution_arn = create_solution_response['solutionArn']
print(json.dumps(create_solution_response, indent=2))

#### 솔루션 버전 생성

In [ ]:
create_solution_version_response = personalize.create_solution_version(
    solutionArn = solution_arn
)

solution_version_arn = create_solution_version_response['solutionVersionArn']
print(json.dumps(create_solution_version_response, indent=2))

#### 솔루션 버전이 활성 상태가 될 때까지 기다리기

이 작업에는 40~50분 정도 걸립니다.

In [ ]:
max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_solution_version_response = personalize.describe_solution_version(
        solutionVersionArn = solution_version_arn
    )
    status = describe_solution_version_response["solutionVersion"]["status"]
    print("SolutionVersion: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)

#### 솔루션 버전의 지표 가져오기

이제 솔루션과 버전이 존재하므로 성능을 판단할 지표를 얻을 수 있습니다. 이러한 지표는 데모 데이터 세트이므로 특별히 좋은 것은 아니지만, 데이터 세트가 더 크고 복잡해질수록 향상됩니다.

In [ ]:
get_solution_metrics_response = personalize.get_solution_metrics(
    solutionVersionArn = solution_version_arn
)

print(json.dumps(get_solution_metrics_response, indent=2))

지표를 이해하려면 [설명서](https://docs.aws.amazon.com/personalize/latest/dg/working-with-training-metrics.html)를 읽는 것이 좋지만, 편의를 위해 아래에 설명서의 일부를 복사해 두었습니다.

Personalize에서의 평가와 관련하여 다음 용어를 이해해야 합니다.

- 연관 추천은 특정 사용자에 대한 테스트 데이터의 값과 일치하는 추천을 말합니다.
- 순위는 추천 목록에서 추천 항목의 위치를 나타냅니다. 위치 1(목록 맨 위)은 사용자와 가장 연관성이 높은 것으로 간주됩니다.
- 쿼리는 GetRecommendations 호출에 해당하는 내부 정보를 말합니다.

Personalize에서 생성되는 지표는 다음과 같습니다.

- coverage: 훈련 데이터의 전체 고유 항목 수 중 모든 쿼리의 고유 추천 항목 비율(항목 및 상호 작용 데이터 세트 모두 포함)입니다.
- mean_reciprocal_rank_at_25: 모든 쿼리에 대한 상위 25개 추천 항목 중 첫 번째 관련 추천의 [평균 상호 순위](https://en.wikipedia.org/wiki/Mean_reciprocal_rank)입니다. 이 지표는 순위가 가장 높은 단일 추천 항목을 파악하려는 경우 적합합니다.
- normalized_discounted_cumulative_gain_at_K: 절감 이득은 추천 목록에서 순위가 낮은 추천이 순위가 높은 추천보다 연관성이 적다고 가정합니다. 따라서 각 추천은 해당 위치에 따라 인수에 의해 절감됩니다(낮은 가중치가 부여됨). K에서 [누적 절감 이득](https://en.wikipedia.org/wiki/Discounted_cumulative_gain)(DCG)을 산출하기 위해 상위 K개 추천의 각 연관 절감 추천을 함께 집계합니다. 정규화된 절감 누적 이득(NDCG)은 DCG를 이상적인 DCG로 나누어 NDCG가 0~1이 되도록 합니다. (이상적인 DCG란 상위 K 추천이 연관성에 따라 정렬하는 경우입니다.) Amazon Personalize는 1/log(1 + 위치)라는 가중치 인수를 사용합니다. 여기서 목록의 최상위는 위치 1입니다. 목록의 최상위에 있는 항목이 더 많은 주목을 받게 되므로, 이 지표는 목록 맨 위에 있는 연관 항목에 가중치를 부여합니다.
- precision_at_K: 상위 K개의 추천 중 연관 추천 수를 K로 나눈 값입니다. 이 지표는 연관 항목의 정확한 추천에 대해 가중치를 부여합니다.

## 캠페인 생성 및 기다리기

이제 잘 작동하는 솔루션 버전이 있으므로 애플리케이션에 사용할 캠페인을 만들어야 합니다. 캠페인은 호스팅된 솔루션 버전으로, 추천을 쿼리할 수 있는 엔드포인트입니다. 요금은 처리 용량(초당 사용자의 개인화 요청)을 추정하여 정해집니다. 캠페인을 배포할 때 최소 초당 트랜잭션(TPS) 값(`minProvisionedTPS`)을 설정합니다. 이 서비스는 AWS 내의 다른 서비스와 마찬가지로 수요에 따라 자동으로 확장되지만, 대기 시간이 중요한 경우에는 더 많은 수요에 대비해 미리 프로비저닝하는 것이 좋습니다. 이 데모에서는 최소 처리량 임계값을 1로 설정합니다. 자세한 내용은 [요금](https://aws.amazon.com/personalize/pricing/) 페이지를 참조하세요.

앞서 언급했듯이, 이 솔루션에 사용되는 사용자 개인화 레시피는 "콜드" 항목의 자동 탐색을 지원합니다. 캠페인을 만들 때 수행하는 탐색량을 제어할 수 있습니다. `itemExplorationConfig` 데이터 유형은 `explorationWeight` 및 `explorationItemAgeCutOff` 파라미터를 지원합니다. 탐색 가중치는 상호 작용 데이터 또는 연관성이 적은 항목을 추천에 얼마나 자주 포함할지를 결정합니다. 값이 1.0에 가까울수록 탐색량이 늘어납니다. 0인 경우 탐색이 발생하지 않으며 추천이 현재 데이터(연관성)를 기반으로 합니다. 탐색 항목 보존 기간 컷오프는 최근 상호 작용 이후 기간을 기준으로 탐색할 항목을 결정합니다. 항목 탐색 범위를 정의하기 위해 최근 상호 작용 이후 최대 항목 보존 기간(일)을 제공합니다. 이 값이 클수록 탐색 시에 더 많은 항목이 고려됩니다. 아래의 캠페인에서는 탐색 가중치를 0.5로 지정합니다.

#### 캠페인 생성

In [ ]:
create_campaign_response = personalize.create_campaign(
    name = "personalize-demo-camp",
    solutionVersionArn = solution_version_arn,
    minProvisionedTPS = 1,
    campaignConfig = {
        "itemExplorationConfig": {
            "explorationWeight": "0.5"
        }
    }
)

campaign_arn = create_campaign_response['campaignArn']
print(json.dumps(create_campaign_response, indent=2))

#### 캠페인이 활성 상태가 될 때까지 기다리기

활성 상태가 되는 데 10분 정도 걸립니다.

In [ ]:
max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_campaign_response = personalize.describe_campaign(
        campaignArn = campaign_arn
    )
    status = describe_campaign_response["campaign"]["status"]
    print("Campaign: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)

## 샘플 추천 받기

캠페인이 활성 상태가 되면 추천을 받을 준비가 된 것입니다. 먼저 컬렉션에서 임의의 사용자를 선택해야 합니다. 그런 다음 ID뿐만 아니라, 추천을 위한 다른 영화 정보도 얻을 수 있도록 몇 가지 헬퍼 함수를 만듭니다.

In [ ]:
# Getting a random user:
user_id, item_id, _ = data.sample().values[0]
print("USER: {}".format(user_id))

In [ ]:
# First load items into memory
items = pd.read_csv('./ml-100k/u.item', sep='|', usecols=[0,1], encoding='latin-1', names=['ITEM_ID', 'TITLE'], index_col='ITEM_ID')

def get_movie_title(movie_id):
    """
    Takes in an ID, returns a title
    """
    movie_id = int(movie_id)-1
    return items.iloc[movie_id]['TITLE']


#### GetRecommendations 호출

위에서 얻은 사용자를 사용하여 아래의 코드 줄에서 추천을 받고 추천 영화 목록을 반환합니다.


In [ ]:
get_recommendations_response = personalize_runtime.get_recommendations(
    campaignArn = campaign_arn,
    userId = str(user_id),
)
# Update DF rendering
pd.set_option('display.max_rows', 30)

print("Recommendations for user: ", user_id)

item_list = get_recommendations_response['itemList']

recommendation_list = []

for item in item_list:
    title = get_movie_title(item['itemId'])
    recommendation_list.append(title)
    
recommendations_df = pd.DataFrame(recommendation_list, columns = ['OriginalRecs'])
recommendations_df

## 검토

위의 코드를 사용하여 딥 러닝 모델을 성공적으로 훈련하여 이전 사용자 행동을 기반으로 한 영화 추천을 생성했습니다. 이 데이터를 사용할 수 있는 다른 유형의 문제와 이러한 추천을 제공하기 위해 이와 같은 시스템을 구축하려면 어떻게 해야 할지 생각해보세요.

이제 다음 노트북 `2.View_Campaign_And_Interactions.ipynb`로 넘어갈 준비가 되었습니다.



## 다음 노트북에 대한 참고 사항:

다음 노트북에 필요한 몇 가지 값이 있습니다. 아래 셀을 실행하여, 연습의 다음 부분에서 복사해 붙여 넣을 수 있도록 해당 값을 저장하세요.

In [ ]:
%store campaign_arn

In [ ]:
%store dataset_group_arn

In [ ]:
%store solution_version_arn

In [ ]:
%store solution_arn

In [ ]:
%store dataset_arn

In [ ]:
%store campaign_arn

In [ ]:
%store schema_arn

In [ ]:
%store bucket

In [ ]:
%store filename

In [ ]:
%store role_name

In [ ]:
%store recommendations_df

In [ ]:
%store user_id